In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from keras.utils import to_categorical

2024-05-03 17:16:44.659868: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 17:16:44.766402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 17:16:44.766467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 17:16:44.780502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 17:16:44.826515: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(male)/outfit(male).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-30.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일', '체중', '키', '스타일']].copy()
df_weather = df_weather[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '강수 계속시간(hr)', '평균 풍속(m/s)', '평균 상대습도(%)', '일시']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_weather['일시'] = pd.to_datetime(df_weather['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_weather, left_on='작성일', right_on='일시')
df_merged = df_merged.drop(['작성일', '일시'], axis=1)
df_merged

,userId,상의,아우터,하의,신발,액세서리,체중,키,스타일,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),평균 풍속(m/s),평균 상대습도(%)
0,1,"반팔 티, 셔츠/블라우스",재킷,반바지,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",13.2,11.0,16.5,7.00,3.0,80.1
1,1,반팔 티,재킷,반바지,운동화,기타 모자,67,172,"캐주얼, 프렌치시크",17.6,11.5,24.3,NaN,2.1,51.8
2,1,반팔 티,재킷,반바지,구두/로퍼,장목양말,67,172,"캐주얼, 프렌치시크",16.0,11.7,20.0,12.17,2.5,77.4
3,1,반팔 티,NaN,나일론 팬츠,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",15.3,10.6,20.8,NaN,3.5,32.4
4,1,반팔 티,집업,면바지,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",14.0,10.2,18.8,NaN,2.5,51.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,14,반팔 티,NaN,반바지,운동화,"기타 모자, 장목양말",65,178,캐주얼,17.3,13.2,24.2,3.33,3.6,55.0
1334,14,반팔 티,NaN,카고바지,운동화,기타 모자,65,178,캐주얼,13.2,11.0,16.5,7.00,3.0,80.1
1335,14,반팔 티,집업,나일론 팬츠,스니커즈/캔버스,기타 모자,65,178,캐주얼,14.4,10.3,20.8,NaN,1.9,75.6
1336,14,"반팔 티, 셔츠/블라우스",NaN,반바지,구두/로퍼,장목양말,65,178,캐주얼,17.8,11.4,26.0,NaN,1.9,61.1


In [4]:
# 강수 계속시간을 비 옴, 비 오지 않음으로 변경
df_merged_sky = df_merged.copy()
df_merged_sky['강수 여부'] = np.where(df_merged_sky['강수 계속시간(hr)'] > 0, 1, 0)
df_merged_sky.drop(['강수 계속시간(hr)'], axis=1, inplace=True)
df_merged_sky

,userId,상의,아우터,하의,신발,액세서리,체중,키,스타일,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 상대습도(%),강수 여부
0,1,"반팔 티, 셔츠/블라우스",재킷,반바지,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",13.2,11.0,16.5,3.0,80.1,1
1,1,반팔 티,재킷,반바지,운동화,기타 모자,67,172,"캐주얼, 프렌치시크",17.6,11.5,24.3,2.1,51.8,0
2,1,반팔 티,재킷,반바지,구두/로퍼,장목양말,67,172,"캐주얼, 프렌치시크",16.0,11.7,20.0,2.5,77.4,1
3,1,반팔 티,NaN,나일론 팬츠,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",15.3,10.6,20.8,3.5,32.4,0
4,1,반팔 티,집업,면바지,구두/로퍼,NaN,67,172,"캐주얼, 프렌치시크",14.0,10.2,18.8,2.5,51.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,14,반팔 티,NaN,반바지,운동화,"기타 모자, 장목양말",65,178,캐주얼,17.3,13.2,24.2,3.6,55.0,1
1334,14,반팔 티,NaN,카고바지,운동화,기타 모자,65,178,캐주얼,13.2,11.0,16.5,3.0,80.1,1
1335,14,반팔 티,집업,나일론 팬츠,스니커즈/캔버스,기타 모자,65,178,캐주얼,14.4,10.3,20.8,1.9,75.6,0
1336,14,"반팔 티, 셔츠/블라우스",NaN,반바지,구두/로퍼,장목양말,65,178,캐주얼,17.8,11.4,26.0,1.9,61.1,0


In [6]:
# '상의', '아우터', '하의', '신발', '액세서리' 열의 결측값을 '~ 없음'으로 대체
columns = ['상의', '아우터', '하의', '신발', '액세서리']
df_notnull = df_merged_sky.copy()
for column in columns:
    df_notnull[column] = df_merged[column].fillna(column + ' 없음')
df_notnull['평균 풍속(m/s)'].fillna(df_merged_sky['평균 풍속(m/s)'].mean(), inplace=True)
df_notnull.isna().sum()

/tmp/ipykernel_50068/235919209.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_notnull['평균 풍속(m/s)'].fillna(df_merged_sky['평균 풍속(m/s)'].mean(), inplace=True)


userId        0
상의            0
아우터           0
하의            0
신발            0
액세서리          0
체중            0
키             0
스타일           0
평균기온(°C)      0
최저기온(°C)      0
최고기온(°C)      0
평균 풍속(m/s)    0
평균 상대습도(%)    0
강수 여부         0
dtype: int64

In [7]:
# 2가 붙은 단어를 두 번 반복하는 함수
def duplicate_word(text):
    words = text.split(', ')
    for i, word in enumerate(words):
        if '2' in word:
            words[i] = word.replace('2', '') + ', ' + word.replace('2', '')
    return ', '.join(words)

In [8]:
# 2가 붙은 단어를 두 번 반복한 dataframe df_dup 생성
df_dup = df_notnull
for column in columns:
    df_dup[columns] = df_notnull[columns].map(duplicate_word)

In [9]:
'''df_dup.iloc[168]'''

'df_dup.iloc[168]'

In [11]:
# 옷의 조합 컬럼 생성 (상의, 아우터, 하의, 신발, 액세서리의 각 값들을 하나의 문자열로 조합하여 하나의 컬럼으로 만듦)
df_combination = df_dup.copy()
df_combination['옷 조합'] = df_dup['상의'] + ', ' + df_dup['아우터'] + ', ' + df_dup['하의'] + ', ' + df_dup['신발'] + ', ' + df_dup['액세서리']
df_combination.drop(columns=['상의', '아우터', '하의', '신발', '액세서리'], inplace=True)

In [12]:
# 옷의 조합 컬럼의 공백 제거
df_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')

In [13]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(',')

O_vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

O = O_vectorizer.fit_transform(df_combination['옷 조합'])

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded_O = pd.DataFrame(O.toarray().tolist(), columns=O_vectorizer.get_feature_names_out())
npa_O = np.array(df_encoded_O)
npa_O

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
'''# multi-hot encoding된 데이터를 다시 텍스트로 변환
df_encoded_O = df_combination.copy()
df_encoded_O['옷 조합'] = O_vectorizer.inverse_transform(npa_O)
df_string_O = df_encoded_O.copy()
df_string_O['옷 조합'] = df_encoded_O['옷 조합'].apply(lambda x: ', '.join(map(str, x)))
df_string_O['옷 조합']'''

"# multi-hot encoding된 데이터를 다시 텍스트로 변환\ndf_encoded_O = df_combination.copy()\ndf_encoded_O['옷 조합'] = O_vectorizer.inverse_transform(npa_O)\ndf_string_O = df_encoded_O.copy()\ndf_string_O['옷 조합'] = df_encoded_O['옷 조합'].apply(lambda x: ', '.join(map(str, x)))\ndf_string_O['옷 조합']"

In [16]:
'''unique_values = df_string_O['옷 조합'].unique()
unique_values'''

"unique_values = df_string_O['옷 조합'].unique()\nunique_values"

In [17]:
'''# 각 고유값을 범주형으로 매핑하는 딕셔너리 생성
category_mapping = {value: i for i, value in enumerate(unique_values)}
df_class_O = df_string_O['옷 조합'].map(category_mapping).astype('category').copy()'''

"# 각 고유값을 범주형으로 매핑하는 딕셔너리 생성\ncategory_mapping = {value: i for i, value in enumerate(unique_values)}\ndf_class_O = df_string_O['옷 조합'].map(category_mapping).astype('category').copy()"

In [18]:
# 스타일 칼럼의 공백 제거
df_combination['스타일'] = df_combination['스타일'].str.replace(' ', '')

In [19]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(',')

S_vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

S = S_vectorizer.fit_transform(df_combination['스타일'])

# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded_S = pd.DataFrame(S.toarray().tolist(), columns=S_vectorizer.get_feature_names_out())
df_encoded_S
df_encoded = pd.concat([df_combination.drop(columns=['옷 조합', '스타일']), df_encoded_S, df_encoded_O], axis=1)
df_encoded

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,userId,체중,키,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 상대습도(%),강수 여부,댄디,...,집업,카고바지,코트,털모자,트레이닝/조거팬츠,패딩,패딩슈즈,패딩조끼,하의없음,후드티
0,1,67,172,13.2,11.0,16.5,3.0,80.1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,67,172,17.6,11.5,24.3,2.1,51.8,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,67,172,16.0,11.7,20.0,2.5,77.4,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,67,172,15.3,10.6,20.8,3.5,32.4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,67,172,14.0,10.2,18.8,2.5,51.3,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,14,65,178,17.3,13.2,24.2,3.6,55.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1334,14,65,178,13.2,11.0,16.5,3.0,80.1,1,0,...,0,1,0,0,0,0,0,0,0,0
1335,14,65,178,14.4,10.3,20.8,1.9,75.6,0,0,...,1,0,0,0,0,0,0,0,0,0
1336,14,65,178,17.8,11.4,26.0,1.9,61.1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
labels_columns = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']
inputs_coulumns = ['체중', '키', '평균 풍속(m/s)', '평균 상대습도(%)']

label_scaler = StandardScaler()
input_scaler = StandardScaler()

df_stand_i = df_encoded[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']].copy()
df_stand_l = df_encoded[['체중', '키', '평균 풍속(m/s)', '평균 상대습도(%)']].copy()

df_stand = df_encoded.copy()

df_stand[labels_columns] = label_scaler.fit_transform(df_encoded[labels_columns])
df_stand[inputs_coulumns] = input_scaler.fit_transform(df_encoded[inputs_coulumns])

In [21]:
df_input = df_stand.drop(['userId','평균기온(°C)', '최저기온(°C)', '최고기온(°C)'], axis=1).copy()
df_input.isna().sum()

체중            0
키             0
평균 풍속(m/s)    0
평균 상대습도(%)    0
강수 여부         0
댄디            0
미니멀           0
밀리터리          0
스트릿           0
아메카지          0
아이비리그         0
캐주얼           0
클래식           0
프레피           0
프렌치시크         0
가디건           0
가죽바지          0
구두/로퍼         0
기타모자          0
긴팔티           0
나일론팬츠         0
니트            0
데님팬츠          0
레더부츠          0
레인부츠          0
마스크           0
맨투맨           0
머플러           0
면바지           0
민소매티          0
바람막이          0
반바지           0
반팔            0
반팔니트          0
반팔셔츠/블라우스     0
반팔티           0
비니            0
샌들/슬리퍼        0
셔츠/블라우스       0
스니커즈/캔버스      0
스카프           0
슬랙스           0
아우터없음         0
액세서리없음        0
양말            0
운동화           0
장목양말          0
재킷            0
점퍼            0
조끼            0
집업            0
카고바지          0
코트            0
털모자           0
트레이닝/조거팬츠     0
패딩            0
패딩슈즈          0
패딩조끼          0
하의없음          0
후드티           0
dtype: int64

In [22]:
df_label = df_stand[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']].copy()
df_label

,평균기온(°C),최저기온(°C),최고기온(°C)
0,-0.105343,0.074868,-0.242682
1,0.335939,0.123847,0.536906
2,0.175472,0.143439,0.107133
3,0.105269,0.035684,0.187091
4,-0.025110,-0.003500,-0.012803
...,...,...,...
1333,0.305851,0.290378,0.526911
1334,-0.105343,0.074868,-0.242682
1335,0.015006,0.006296,0.187091
1336,0.355997,0.114051,0.706816


In [23]:
X = np.array(df_input)
Y = np.array(df_label)

In [24]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
X_train

(1070, 60)


array([[ 0.38148917,  1.06312749,  0.46548685, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46499924,  0.36675123,  3.24793189, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46499924,  0.36675123,  0.31090657, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.46499924,  0.36675123, -0.30741455, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17040624, -1.49025214, -0.61657511, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46499924,  0.36675123, -0.61657511, ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
print(Y_train.shape)
Y_train

(1070, 3)


array([[-0.76726676, -0.92431612, -0.75241229],
       [-1.84038483, -1.69819383, -1.86182564],
       [ 0.97780376,  0.68221485,  1.22654124],
       ...,
       [-1.00796614, -1.17901005, -0.83237001],
       [ 1.48928995,  1.49527625,  1.43643026],
       [-0.13543088, -0.50309153,  0.10713318]])

In [26]:
input = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(512, activation='relu')(input)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(Y_train.shape[1])(x)

2024-05-03 17:17:30.945255: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:17:31.106952: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:17:31.107009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:17:31.111866: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:17:31.111943: I external/local_xla/xla/stream_executor

In [27]:
model = keras.Model(input, output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [28]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_val, Y_val))

Epoch 1/20


2024-05-03 17:17:34.792955: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-03 17:17:35.591252: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fcab97f3960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-03 17:17:35.591304: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-05-03 17:17:35.613733: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 17:17:35.657401: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1714724255.748513  163222 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/34 [==============================] - 3s 12ms/step - loss: 0.8646 - mae: 0.7838 - val_loss: 0.5821 - val_mae: 0.6142
Epoch 2/20
34/34 [==============================] - 0s 8ms/step - loss: 0.5675 - mae: 0.6147 - val_loss: 0.4636 - val_mae: 0.5427
Epoch 3/20
34/34 [==============================] - 0s 7ms/step - loss: 0.4819 - mae: 0.5588 - val_loss: 0.5251 - val_mae: 0.5861
Epoch 4/20
34/34 [==============================] - 0s 7ms/step - loss: 0.4721 - mae: 0.5587 - val_loss: 0.4138 - val_mae: 0.5234
Epoch 5/20
34/34 [==============================] - 0s 8ms/step - loss: 0.4529 - mae: 0.5368 - val_loss: 0.4606 - val_mae: 0.5498
Epoch 6/20
34/34 [==============================] - 0s 8ms/step - loss: 0.4145 - mae: 0.5185 - val_loss: 0.4678 - val_mae: 0.5516
Epoch 7/20
34/34 [==============================] - 0s 8ms/step - loss: 0.3917 - mae: 0.4986 - val_loss: 0.3983 - val_mae: 0.5102
Epoch 8/20
34/34 [==============================] - 0s 8ms/step - loss: 0.3777 - mae: 0.4906 - val_l

In [29]:
result = model.predict(X_val[4:5])
print(f'예측 결과: {label_scaler.inverse_transform(result)}')
print(f'실제 데이터: {input_scaler.inverse_transform(X_val[4:5, 1:5])}, {X_val[4:5, 5:]} 실제 결과: {label_scaler.inverse_transform(Y_val[4:5])} ')

1/1 [==============================] - 0s 101ms/step
예측 결과: [[14.176419  8.985228 20.338335]]
실제 데이터: [[ 74.1861953  173.76381454   1.11578723  66.61307922]], [[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]] 실제 결과: [[10.6  3.4 17.7]] 
